In [9]:
import pandas as pd

import os
# import sys
# import wave
# import copy
# import math
# import tensorflow as tf
from keras.models import Sequential, Model, load_model
# from keras.layers.core import Dense, Activation
# # from keras.layers import LSTM, Input, Flatten, Merge, Bidirectional
# from keras.layers import LSTM, Input, Flatten, Bidirectional
# from sklearn.preprocessing import label_binarize

from features import *
from helper import *
import pyaudio
import wave
import numpy as np
import struct
import sys
import matplotlib.pyplot as plt

%matplotlib tk
#%matplotlib inline

# constants

TIME_SEC = 1
RATE = 44100                 # samples per second
CHUNK = 44100 * TIME_SEC #1024 * 20 #int(44100 / TIME_SEC)             # samples per frame
FORMAT = pyaudio.paInt16     # audio format (bytes per sample?)
CHANNELS = 1                 # single channel for microphone


p = pyaudio.PyAudio()
for i in range(p.get_device_count()):
    print(p.get_device_info_by_index(i))

Using TensorFlow backend.


{'index': 0, 'structVersion': 2, 'name': 'Microsoft Sound Mapper - Input', 'hostApi': 0, 'maxInputChannels': 2, 'maxOutputChannels': 0, 'defaultLowInputLatency': 0.09, 'defaultLowOutputLatency': 0.09, 'defaultHighInputLatency': 0.18, 'defaultHighOutputLatency': 0.18, 'defaultSampleRate': 44100.0}
{'index': 1, 'structVersion': 2, 'name': 'Microphone (Realtek(R) Audio)', 'hostApi': 0, 'maxInputChannels': 4, 'maxOutputChannels': 0, 'defaultLowInputLatency': 0.09, 'defaultLowOutputLatency': 0.09, 'defaultHighInputLatency': 0.18, 'defaultHighOutputLatency': 0.18, 'defaultSampleRate': 44100.0}
{'index': 2, 'structVersion': 2, 'name': 'Microsoft Sound Mapper - Output', 'hostApi': 0, 'maxInputChannels': 0, 'maxOutputChannels': 2, 'defaultLowInputLatency': 0.09, 'defaultLowOutputLatency': 0.09, 'defaultHighInputLatency': 0.18, 'defaultHighOutputLatency': 0.18, 'defaultSampleRate': 44100.0}
{'index': 3, 'structVersion': 2, 'name': 'Speakers (Realtek(R) Audio)', 'hostApi': 0, 'maxInputChannels': 

In [2]:
# create matplotlib figure and axes
#fig, ax = plt.subplots(1, figsize=(15, 7))

# pyaudio class instance
p = pyaudio.PyAudio()

import sounddevice as sd
print(sd.query_devices())

print("starting stream declaration")

# stream object to get data from microphone
stream = p.open(
    format=FORMAT,
    channels=CHANNELS,
    rate=RATE,
    input=True,
    output=True,
    frames_per_buffer=CHUNK,
    input_device_index = 1
)

   0 Microsoft Sound Mapper - Input, MME (2 in, 0 out)
>  1 Microphone (Realtek(R) Audio), MME (2 in, 0 out)
   2 Microsoft Sound Mapper - Output, MME (0 in, 2 out)
<  3 Speakers (Realtek(R) Audio), MME (0 in, 2 out)
   4 BenQ GW2480 (Intel(R) Display A, MME (0 in, 2 out)
   5 Primary Sound Capture Driver, Windows DirectSound (2 in, 0 out)
   6 Microphone (Realtek(R) Audio), Windows DirectSound (2 in, 0 out)
   7 Primary Sound Driver, Windows DirectSound (0 in, 2 out)
   8 Speakers (Realtek(R) Audio), Windows DirectSound (0 in, 2 out)
   9 BenQ GW2480 (Intel(R) Display Audio), Windows DirectSound (0 in, 2 out)
  10 BenQ GW2480 (Intel(R) Display Audio), Windows WASAPI (0 in, 2 out)
  11 Speakers (Realtek(R) Audio), Windows WASAPI (0 in, 2 out)
  12 Microphone (Realtek(R) Audio), Windows WASAPI (2 in, 0 out)
  13 Speakers (Realtek HD Audio output), Windows WDM-KS (0 in, 8 out)
  14 Microphone 1 (Realtek HD Audio Mic input with SST), Windows WDM-KS (2 in, 0 out)
  15 Microphone 2 (Realtek

In [5]:
# model = load_model("Trained_models/2_layer_LSTM.pickle")

In [10]:
# define callback (2)
def callback(in_data, frame_count, time_info, status):
    data = wf.readframes(frame_count)
    # data_int = struct.unpack(str(2 * CHUNK) + 'B', data)
    with open(LOG_PATH + "log_file.txt", "a") as f:
            f.write(str(data) + "\n")

    return (data, pyaudio.paContinue)

In [11]:
wf = wave.open(WAV_PATH, 'rb')

# instantiate PyAudio (1)
p = pyaudio.PyAudio()

# open stream using callback (3)
stream = p.open(format=p.get_format_from_width(wf.getsampwidth()),
                channels=wf.getnchannels(),
                rate=wf.getframerate(),
                input = True,
                stream_callback=callback)

In [ ]:
# start the stream (4)
stream.start_stream()


In [12]:
# stop stream (6)
stream.stop_stream()
stream.close()
wf.close()

# close PyAudio (7)
p.terminate()